# Создание таблиц

In [12]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS customer_20240101 (
        customer_id INT PRIMARY KEY,
        first_name VARCHAR(50),
        last_name VARCHAR(50),
        gender VARCHAR(30),
        dob VARCHAR(50),
        job_title VARCHAR(50),
        job_industry_category VARCHAR(50),
        wealth_segment VARCHAR(50),
        deceased_indicator VARCHAR(50),
        owns_car VARCHAR(30),
        address VARCHAR(50),
        postcode VARCHAR(30),
        state VARCHAR(30),
        country VARCHAR(30),
        property_valuation INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS transaction_20240101 (
        transaction_id INT PRIMARY KEY,
        product_id INT,
        customer_id INT REFERENCES customer_20240101(customer_id) ON DELETE CASCADE,
        transaction_date VARCHAR(30),
        online_order VARCHAR(30),
        order_status VARCHAR(30),
        brand VARCHAR(30),
        product_line VARCHAR(30),
        product_class VARCHAR(30),
        product_size VARCHAR(30),
        list_price FLOAT,
        standard_cost FLOAT
    )
''')

conn.commit()

def load_csv_to_db(csv_path, table_name, skip_invalid=False):
    df = pd.read_csv(csv_path, sep=';', decimal=',')
    df.columns = [col.strip() for col in df.columns]
    if 'transaction_date' in df.columns:
        df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')

    if table_name == "transaction_20240101":
        cursor.execute("SELECT customer_id FROM customer_20240101")
        valid_customers = {row[0] for row in cursor.fetchall()}
        
        df = df[df["customer_id"].isin(valid_customers)]
    
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join(['%s'] * len(row))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values}) ON CONFLICT DO NOTHING"
        cursor.execute(insert_query, tuple(row))
    
    conn.commit()

load_csv_to_db('customer.csv', 'customer_20240101')
load_csv_to_db('transaction.csv', 'transaction_20240101')

cursor.close()
conn.close()


# Создание запросов

In [14]:

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

def execute_query(query):
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    print(df)
    return df

query_1 = """
    SELECT DISTINCT brand
    FROM transaction_20240101
    WHERE CAST(standard_cost AS FLOAT) > 1500
    AND brand <> 'NaN';
"""
print("1) Уникальные бренды с cost > 1500:")
execute_query(query_1)

query_2 = """
    SELECT * 
FROM transaction_20240101
WHERE order_status = 'Approved'
AND transaction_date BETWEEN '2017-04-01' AND '2017-04-09';
"""
print("\n2) Подтвержденные транзакции 2017-04-01 — 2017-04-09:")
execute_query(query_2)

query_3 = """
    SELECT DISTINCT job_title
    FROM customer_20240101
    WHERE job_industry_category IN ('IT', 'Financial Services')
    AND job_title ILIKE 'Senior%';
"""
print("\n3) Профессии клиентов из IT или Financial Services, начинающиеся с 'Senior':")
execute_query(query_3)

query_4 = """
    SELECT DISTINCT t.brand
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    WHERE c.job_industry_category = 'Financial Services'
    AND brand <> 'NaN';
"""
print("\n4) Бренды, закупаемые клиентами из Financial Services:")
execute_query(query_4)

query_5 = """
    SELECT DISTINCT c.customer_id, c.first_name, c.last_name
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    WHERE LOWER(t.online_order) = 'true'
    AND t.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
    LIMIT 10;
"""
print("\n5) 10 клиентов, оформивших онлайн-заказ у указанных брендов:")
execute_query(query_5)

query_6 = """
    SELECT c.customer_id, c.first_name, c.last_name
    FROM customer_20240101 c
    LEFT JOIN transaction_20240101 t ON c.customer_id = t.customer_id
    WHERE t.customer_id IS NULL;
"""
print("\n6) Клиенты без транзакций:")
execute_query(query_6)

# Данные не полные, для IT нет данных по стандартной стоимости
query_7 = """
    SELECT c.customer_id, c.first_name, c.last_name, t.standard_cost::FLOAT
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    WHERE c.job_industry_category = 'IT'
    AND t.standard_cost::FLOAT = (SELECT MAX(standard_cost::FLOAT) FROM transaction_20240101);
"""
print("\n7) Клиенты из IT с максимальной стандартной стоимостью транзакции:")
execute_query(query_7)

query_8 = """
    SELECT DISTINCT c.customer_id, c.first_name, c.last_name
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    WHERE c.job_industry_category IN ('IT', 'Health')
    AND t.order_status = 'Approved'
    AND transaction_date BETWEEN '2017-07-07' AND '2017-07-17';

"""
print("\n8) Клиенты из IT и Health с подтвержденными транзакциями 2017-07-07 — 2017-07-17:")
execute_query(query_8)

cursor.close()
conn.close()


1) Уникальные бренды с cost > 1500:
            brand
0      OHM Cycles
1   Trek Bicycles
2           Solex
3  Giant Bicycles

2) Подтвержденные транзакции 2017-04-01 — 2017-04-09:
     transaction_id  product_id  customer_id transaction_date online_order  \
0                17          79         2426       2017-04-03        false   
1                19          54         2268       2017-04-06         true   
2                23          37         2001       2017-04-08         true   
3                83           0         3398       2017-04-01         true   
4                89           0         2682       2017-04-04         true   
..              ...         ...          ...              ...          ...   
526           19655           0          336       2017-04-09         true   
527           19853           7         3072       2017-04-02        false   
528           19899          57          325       2017-04-06        false   
529           19968           0        